In [1]:
from datetime import datetime
import glob
import re
import json
import os.path
import time
import re
from nltk.tokenize import RegexpTokenizer
from nltk.stem import SnowballStemmer
import collections
import os
from math import log
from math import exp

In [2]:
data_set = []

In [3]:
len(data_set)

0

In [4]:
paths = glob.glob("/home/parth/information_retrieval/Data/AP89_DATA/AP_DATA/ap89_collection/*")

In [5]:
x = 0
doc_total = 0
for path in paths:
    try:
        fnames = path.split("/")[-1]
        f = open(path,"r", encoding='iso-8859-1')
        text = f.read()
        doc_total = doc_total + text.count('<DOC>')
        for m in zip(re.finditer('<DOC>', text), re.finditer('</DOC>', text)):
            docdata = text[m[0].start():m[1].end()]
            file_data = {}
            for m in zip(re.finditer('<DOCNO>', docdata), re.finditer('</DOCNO>', docdata)):
                doc_no = docdata[m[0].end() + 1 : m[1].start()]
                file_data['DOCNO'] = doc_no

            file_data['TEXT'] = ""
            for m in zip(re.finditer('<TEXT>', docdata), re.finditer('</TEXT>', docdata)):
                file_data['TEXT'] = file_data['TEXT'] + docdata[m[0].end() + 1: m[1].start()]
#             print(file_data)
            data_set.append(file_data)
    except:
        print(path + " error in file generation")

In [6]:
stop_files = open("stoplist.txt", "r")

In [7]:
tokenizer = RegexpTokenizer(r'[A-Za-z0-9]+')
snowball_stemmer = SnowballStemmer("english")

In [8]:
stopwords = [snowball_stemmer.stem(stopword) for stopword in stop_files.read().split("\n")]

In [9]:
numbers_checker = re.compile(r"(^[\d][\d+\.]+$)")
numbers_checker_comma = re.compile(r"^[\d][\d+\.\,]+$")

In [10]:
def jsontodocencode(jsondata, flname):
    json_offset = {}
    offset = 0
    mainstring = ""
    for term in jsondata.keys():
        string = ""
        string = string + str(term) + " " + str(jsondata[term]['ttf']) + " " + str(jsondata[term]['df']) + " "
        for doc in jsondata[term]['tf'].keys():
            string = string + str(doc) + " " + str(jsondata[term]['tf'][doc]['tf']) + " "
            for pos in jsondata[term]['tf'][doc]['position']:
                string = string + str(pos) + " "

        
        string = string + "\n"
        json_offset[term] = offset
        offset = offset + len(string)
        mainstring = mainstring + string

    with open(flname + 'catlog' +'.json', 'w') as outfile:
        json.dump(json_offset, outfile)
    
    with open(flname, "w") as output:
        output.write(mainstring)
        


In [11]:
start_time = time.time()
start = 0
docid = 0
docmap = {}
reversedocmap = {}

index = 0
vocabmap = {}
vocab = {}
reverse_vocab = {}
doc_length_dict = {}


for i in range(1, int(len(data_set) / 1000) + 1):
# for i in range(1,int(len(data_set_test)/2)+1):
    fname = "index/tokenisation_" + str(start) + "_" + str(i*1000)
    inverted_index = {}
    file  = open(fname, "w")

    for doc in data_set[start:(i*1000) + 1]:
#     for doc in data_set_test[start:(i*2) + 1]:
        docmap[doc['DOCNO']] = docid
        reversedocmap[docid] = doc['DOCNO']
        pos = 0
        doc_length_dict[docid] = 0
        for token in doc['TEXT'].split():
            token = token.lower()
            if(len(re.findall(numbers_checker, token)) == 1):
                token = token
            elif(len(re.findall(numbers_checker_comma,token)) == 1):
                token = token.replace(",","")
            else:
                token = tokenizer.tokenize(token)
                if(len(token) > 0):
                    token = token[0]
                else:
                    token = ""
                token = snowball_stemmer.stem(token)

            if token not in stopwords :
                if token not in vocab.keys():
                    vocab[token] = 0 
                    vocabmap[token] = index
                    reverse_vocab[index] = token
                    index = index + 1

                doc_length_dict[docid] = doc_length_dict[docid] + 1
                token_id = vocabmap[token]
                if token_id not in inverted_index.keys():
                    inverted_index[token_id] = {
                        "ttf" : 0,
                        "df" : 0,
                        "tf" : {}
                    }
                inverted_index[token_id]["ttf"] = inverted_index[token_id]["ttf"] + 1
                if(docid not in inverted_index[token_id]['tf'].keys()):
                    inverted_index[token_id]['df'] = inverted_index[token_id]['df'] + 1
                    inverted_index[token_id]['tf'][docid] = {
                        "tf" : 1,
                        "position" : [pos]
                    }
                else:
                    inverted_index[token_id]['tf'][docid]['tf'] =  inverted_index[token_id]['tf'][docid]['tf'] + 1
                    inverted_index[token_id]['tf'][docid]['position'].append(pos)
                vocab[token] = vocab[token] + 1
                file.write("(" + str(vocabmap[token]) + " " + str(docid) + " " + str(pos) + ") " )
                pos = pos + 1            
        docid = docid + 1
    
    with open('inverted_index/' +  str(start) + "_" + str(i*1000) + '.json', 'w') as outfile:
        json.dump(inverted_index, outfile)

    flname = 'inverted_index_text/' +  str(start) + "_" + str(i*1000)
    jsontodocencode(inverted_index, flname)

    start = i * 1000 + 1
    
    
#     break
#     start = i * 2 + 1
    
fname = "index/tokenisation_" + str(start) + "_" + str(len(data_set))
inverted_index = {}

file  = open(fname, "w")
for doc in data_set[start:]:
    docmap[doc['DOCNO']] = docid
    reversedocmap[docid] = doc['DOCNO']
    doc_length_dict[docid] = 0
    pos = 0
    for token in doc['TEXT'].split():
        token = token.lower()
        if(len(re.findall(numbers_checker, token)) == 1):
            token = token
        elif(len(re.findall(numbers_checker_comma,token)) == 1):
            token = token.replace(",","")
        else:
            token = tokenizer.tokenize(token)
            if(len(token) > 0):
                token = token[0]
            else:
                token = ""
            token = snowball_stemmer.stem(token)

        if token not in stopwords :
            if token not in vocab.keys():
                vocab[token] = 0 
                vocabmap[token] = index
                reverse_vocab[index] = token
                index = index + 1
            
            doc_length_dict[docid] = doc_length_dict[docid] + 1
            token_id = vocabmap[token]
            if token_id not in inverted_index.keys():
                inverted_index[token_id] = {
                    "ttf" : 0,
                    "df" : 0,
                    "tf" : {}
                }
            inverted_index[token_id]["ttf"] = inverted_index[token_id]["ttf"] + 1
            if(docid not in inverted_index[token_id]['tf'].keys()):
                inverted_index[token_id]['df'] = inverted_index[token_id]['df'] + 1
                inverted_index[token_id]['tf'][docid] = {
                    "tf" : 1,
                    "position" : [pos]
                }
            else:
                inverted_index[token_id]['tf'][docid]['tf'] =  inverted_index[token_id]['tf'][docid]['tf'] + 1
                inverted_index[token_id]['tf'][docid]['position'].append(pos)

            vocab[token] = vocab[token] + 1
            file.write("(" + str(vocabmap[token]) + " " + str(docid) + " " + str(pos) + ") " )
            pos = pos + 1
    docid = docid + 1
with open('inverted_index/' +  str(start) + "_" + str(len(data_set)) + '.json', 'w') as outfile:
    json.dump(inverted_index, outfile)

flname = 'inverted_index_text/' +  str(start) + "_" + str(len(data_set))
jsontodocencode(inverted_index, flname)

end_time = time.time()
print(end_time - start_time)

897.8753521442413


In [12]:
len(doc_length_dict)

84678

In [10]:
def mergefiles(fl,catlog):
    path = ""
    processed = []
    
    
    path_merged = '/home/parth/information_retrieval/cs6200-assignment/assignment-2/inverted_index_text/'

        
    tempfile = open(path_merged + "temp" , 'w')
    
    with open(path_merged+ 'merged.json') as f:
        catlog_data = json.load(f)
    
    with open(path + catlog) as f:
        catlog_file = json.load(f)

    mergedfile = open(path_merged + 'merged', 'r')
    file = open(path + fl, 'r')
    
    offset = 0
    new_catlog = {}

    
    for term in catlog_data.keys():
        string = ""
        seek_f1 = catlog_data[term]
        mergedfile.seek(seek_f1)
        line_merged = mergedfile.readline().strip("\n")
        if(term in catlog_file.keys()):
            file.seek(catlog_file[term])
            line_file = file.readline().strip("\n")
            tid_ttf_df_file = line_file.split(" ")[0:3]
            tid_ttf_df_merged = line_merged.split(" ")[0:3]
            tid_ttf_df_new = str(tid_ttf_df_file[0]) + " " + str(int(tid_ttf_df_file[1]) + int(tid_ttf_df_merged[1]))
            tid_ttf_df_new = tid_ttf_df_new + " " + str(int(tid_ttf_df_file[2]) + int(tid_ttf_df_merged[2]))
            merged_file_pointer = len(" ".join(tid_ttf_df_merged))
            old_file_pointer = len(" ".join(tid_ttf_df_file))
            tid_ttf_df_new = tid_ttf_df_new + line_merged[merged_file_pointer:] 
            tid_ttf_df_new = tid_ttf_df_new + line_file[old_file_pointer:] + "\n"
            string = tid_ttf_df_new
        else:
            string = string + line_merged + "\n"
        new_catlog[term] = offset
        offset = offset + len(string)
        tempfile.write(string)
        processed.append(term)
        
    for term in catlog_file.keys():
        if term not in(processed):
            processed.append(term)
            new_catlog[term] = offset
            file.seek(catlog_file[term])
            string = file.readline()
            offset = offset + len(string)
            tempfile.write(string)

    
    mergedfile.close()
    os.remove(path_merged + "merged")
    tempfile.close()
    os.rename(path_merged + "temp", path_merged + "merged")
    
    with open(path_merged + 'merged.json', 'w') as outfile:
        json.dump(new_catlog, outfile)



In [11]:
start_time = time.time()

mergedPaths = glob.glob("/home/parth/information_retrieval/cs6200-assignment/assignment-2/inverted_index_text/*")
for path in mergedPaths:
    if path.find('catlog') == -1 and path.find('merged') == -1 and path.find('temp') == -1:
        print(path)
        mergefiles(path, path+"catlog.json")

end_time = time.time()
print(end_time - start_time)

/home/parth/information_retrieval/cs6200-assignment/assignment-2/inverted_index_text/80001_81000
/home/parth/information_retrieval/cs6200-assignment/assignment-2/inverted_index_text/35001_36000
/home/parth/information_retrieval/cs6200-assignment/assignment-2/inverted_index_text/82001_83000
/home/parth/information_retrieval/cs6200-assignment/assignment-2/inverted_index_text/39001_40000
/home/parth/information_retrieval/cs6200-assignment/assignment-2/inverted_index_text/14001_15000
/home/parth/information_retrieval/cs6200-assignment/assignment-2/inverted_index_text/15001_16000
/home/parth/information_retrieval/cs6200-assignment/assignment-2/inverted_index_text/3001_4000
/home/parth/information_retrieval/cs6200-assignment/assignment-2/inverted_index_text/29001_30000
/home/parth/information_retrieval/cs6200-assignment/assignment-2/inverted_index_text/55001_56000
/home/parth/information_retrieval/cs6200-assignment/assignment-2/inverted_index_text/30001_31000
/home/parth/information_retrieva

1570.9899606704712


In [15]:
def texttojondecode(term):
    result = {}
    path = "/home/parth/information_retrieval/cs6200-assignment/assignment-2/inverted_index_text/"
    
    with open(path+ 'merged.json') as f:
        catlog_data = json.load(f)
        
    mergedfile = open(path + 'merged', 'r')

    seek_f1 = catlog_data[str(term)]
    mergedfile.seek(seek_f1)
    
    result["term_id"] = term
    result["term"] = reverse_vocab[term]
 
    txt = mergedfile.readline().strip("\n").split(" ")
    txt = [int(x) for x in txt if(x is not '')]
    doc_pointer = 3
    doc_f = int(txt[2])
    result['doc_freq'] = doc_f
    result['tt_freq'] = int(txt[1])
    result['doc'] = {}

    for x in range(0,doc_f):
        result['doc'][txt[doc_pointer]] = {
            "tf" : txt[doc_pointer + 1],
            "pos" : []
        } 
#         print("document_id:",  txt[doc_pointer])
#         print("document frequency:",  txt[doc_pointer + 1])
        tf = int(txt[doc_pointer + 1])
        tf_pointer = doc_pointer + 1
        for i in range(1, tf+1):
            result['doc'][txt[doc_pointer]]['pos'].append(txt[tf_pointer + i])
#             print("posistion  :", txt[tf_pointer + i])
        doc_pointer = 1 + (doc_pointer + 1) + int(txt[(doc_pointer + 1)]) 
    return(result)

In [16]:
queriesFile = '/home/parth/information_retrieval/Data/AP89_DATA/AP_DATA/query_desc.51-100.short.txt' 
f = open(queriesFile,"r", encoding='iso-8859-1')
Querytext = f.read()

queries = Querytext.split('\n')[0:-1]
query_dict = {}

for query in queries:
    temp = query.split("   ")
#     query_dict[temp[0].split(".")[0]] = temp[1] 
    tokens = []
    for token in temp[1].split():
        token = token.lower()
        if(len(re.findall(numbers_checker, token)) == 1):
            token = token
        elif(len(re.findall(numbers_checker_comma,token)) == 1):
            token = token.replace(",","")
        else:
            token = tokenizer.tokenize(token)
            if(len(token) > 0):
                token = token[0]
            else:
                token = ""
            token = snowball_stemmer.stem(token)

        if token not in stopwords and token != 'document':
            tokens.append(token)
    query_dict[temp[0].split(".")[0]] = tokens 


In [17]:
total_length = 0 
for doc in doc_length_dict:
    total_length = total_length + doc_length_dict[doc]

In [18]:
avg_len = total_length/len(doc_length_dict)

In [19]:
totaldoc = len(doc_length_dict)

In [20]:
# avg_len

In [21]:
# f_score = {}
# for queryId in query_dict.keys():
#     score = {}
#     for term in query_dict[queryId]:
#         if term in vocabmap.keys():
#             term_id = vocabmap[term]
#             data = texttojondecode(term_id)
#             for doc in data['doc']:
#                 if doc not in score.keys():
#                     score[doc] = 0
#                 _tf = int(data['doc'][doc]['tf']) 
#                 _l = doc_length_dict[doc]
                
#                 _score = _tf / (_tf + 0.5 + (1.5 * (_l/avg_len)))
#                 score[doc] = score[doc] + _score
#     f_score[queryId] = sorted(score.items(), key=lambda kv: kv[1])[::-1][0:1000]


In [22]:
# file = open("okaptf_scores", "w")
# for query in f_score.keys():
#     r = 1
#     for document in f_score[query]:
#         if(document[1] != 0):
#             file.write(str(query) + " Q0 " + reversedocmap[document[0]] + " " + str(r) + " " + str(document[1]) + " Exp\n" )
#         else:
#             print(query)
#         r = r + 1

In [23]:
# f_score = {}
# for queryId in query_dict.keys():
#     score = {}
#     for term in query_dict[queryId]:
#         if term in vocabmap.keys():
#             term_id = vocabmap[term]
#             data = texttojondecode(term_id)
#             for doc in data['doc']:
#                 if doc not in score.keys():
#                     score[doc] = 0
#                 _tf = int(data['doc'][doc]['tf']) 
#                 _l = doc_length_dict[doc]
                
#                 _score = _tf / (_tf + 0.5 + (1.5 * (_l/avg_len)))
#                 _score = _score * log(totaldoc / data['doc_freq']) 
#                 score[doc] = score[doc] + _score
#     f_score[queryId] = sorted(score.items(), key=lambda kv: kv[1])[::-1][0:1000]


In [24]:
# file = open("tfidf_scores", "w")
# for query in f_score.keys():
#     r = 1
#     for document in f_score[query]:
#         if(document[1] != 0):
#             file.write(str(query) + " Q0 " + reversedocmap[document[0]] + " " + str(r) + " " + str(document[1]) + " Exp\n" )
#         else:
#             print(query)
#         r = r + 1

In [25]:
# k1 = 1.2
# k2 = 1000
# b = 0.75

In [26]:
# f_score = {}
# for queryId in query_dict.keys():
#     score = {}
    
#     tfq = collections.Counter(query_dict[queryId])
    
#     for term in query_dict[queryId]:
#         if term in vocabmap.keys():
#             term_id = vocabmap[term]
#             data = texttojondecode(term_id)
#             for doc in data['doc']:
#                 if doc not in score.keys():
#                     score[doc] = 0
#                 _tf = int(data['doc'][doc]['tf']) 
#                 _l = doc_length_dict[doc]
#                 _term_1 = (totaldoc + 0.05) / (data['doc_freq'] + 0.5)
#                 _term_2 = ((1 + k1) * _tf) / (_tf + k1 * ((1-b) + b * ((_l/avg_len))))
#                 _term_3 = tfq[term] * ( 1 + k2) / (tfq[term] + k2)
#                 _score = log(_term_1) * _term_2 * _term_3 
#                 score[doc] = score[doc] + _score
#     f_score[queryId] = sorted(score.items(), key=lambda kv: kv[1])[::-1][0:1000]


In [27]:
# file = open("okapibm25_scores", "w")
# for query in f_score.keys():
#     r = 1
#     for document in f_score[query]:
#         if(document[1] != 0):
#             file.write(str(query) + " Q0 " + reversedocmap[document[0]] + " " + str(r) + " " + str(document[1]) + " Exp\n" )
#         else:
#             print(query)
#         r = r + 1

In [28]:
# V = len(reverse_vocab)

In [29]:
# f_score = {}
# for queryId in query_dict.keys():
#     doc_terms = {}
#     score = {}
#     tfq = collections.Counter(query_dict[queryId])
    
#     for term in query_dict[queryId]:
#         if term in vocabmap.keys():
#             term_id = vocabmap[term]
#             data = texttojondecode(term_id)
#             for doc in data['doc']:
#                 if doc not in score.keys():
#                     score[doc] = 0
#                     doc_terms[doc] = dict(tfq)
#                 doc_terms[doc][term] = doc_terms[doc][term] - 1
#                 _tf = int(data['doc'][doc]['tf']) 
#                 _l = doc_length_dict[doc]
#                 _score = log( (_tf + 1) / (_l + V))
#                 score[doc] = score[doc] + _score
#     for doc in doc_terms.keys():
#         _l = doc_length_dict[doc]
#         for word in doc_terms[doc]:
#             score[doc] = score[doc] + (doc_terms[doc][word] * log(1 / (_l + V)))
#     f_score[queryId] = sorted(score.items(), key=lambda kv: kv[1])[::-1][0:1000]

In [30]:
# file = open("laplace_scores", "w")
# for query in f_score.keys():
#     r = 1
#     for document in f_score[query]:
#         if(document[1] != 0):
#             file.write(str(query) + " Q0 " + reversedocmap[document[0]] + " " + str(r) + " " + str(document[1]) + " Exp\n" )
#         else:
#             print(query)
#         r = r + 1

In [31]:
# lamda = 0.75

In [32]:
# f_score = {}
# for queryId in query_dict.keys():
#     doc_terms = {}
#     score = {}
#     tfq = collections.Counter(query_dict[queryId])
#     ttfs = {}
#     for term in query_dict[queryId]:
#         if term in vocabmap.keys():
#             term_id = vocabmap[term]
#             data = texttojondecode(term_id)
#             for doc in data['doc']:
#                 if doc not in score.keys():
#                     score[doc] = 0
#                     doc_terms[doc] = dict(tfq)
#                 doc_terms[doc][term] = doc_terms[doc][term] - 1
#                 _tf = int(data['doc'][doc]['tf']) 
#                 _l = doc_length_dict[doc]
#                 _ttf = data['tt_freq']
#                 ttfs[term] = _ttf
#                 _pa = (lamda * _tf) /(_l)
#                 _pb = (1 - lamda) * (_ttf / (total_length - _l))
#                 _score = log(_pa + _pb)
#                 score[doc] = score[doc] + _score
#     for doc in doc_terms.keys():
#         _l = doc_length_dict[doc]
#         for word in doc_terms[doc]:
#             score[doc] = score[doc] + (doc_terms[doc][word] * log((1 - lamda) * ttfs[word] / (total_length - _l)))
#     f_score[queryId] = sorted(score.items(), key=lambda kv: kv[1])[::-1][0:1000]

In [33]:
# file = open("laplace_JM_scores", "w")
# for query in f_score.keys():
#     r = 1
#     for document in f_score[query]:
#         if(document[1] != 0):
#             file.write(str(query) + " Q0 " + reversedocmap[document[0]] + " " + str(r) + " " + str(document[1]) + " Exp\n" )
#         else:
#             print(query)
#         r = r + 1

In [34]:
# term_id = vocabmap['government']
# data = texttojondecode(term_id)

In [35]:
# queriesFile = 'modifiedQueries' 
# f = open(queriesFile,"r", encoding='iso-8859-1')
# Querytext = f.read()
# modifiedqueries = Querytext.split('\n')[0:-1]
# modified_query_dict = {}

# for query in modifiedqueries:
#     temp = query.split("   ")
# #     query_dict[temp[0].split(".")[0]] = temp[1] 
#     tokens = []
#     for token in temp[1].split():
#         token = token.lower()
#         if(len(re.findall(numbers_checker, token)) == 1):
#             token = token
#         elif(len(re.findall(numbers_checker_comma,token)) == 1):
#             token = token.replace(",","")
#         else:
#             token = tokenizer.tokenize(token)
#             if(len(token) > 0):
#                 token = token[0]
#             else:
#                 token = ""
#             token = snowball_stemmer.stem(token)

#         if token not in stopwords and token != 'document':
#             tokens.append(token)
#     modified_query_dict[temp[0].split(".")[0]] = tokens 

In [36]:
def findrange(test_range_dict):
    if(len(test_range_dict) == 1) : 
        return 1000
    sorted_pos = {}
    sorted_pointers = {}
    smallest_word = ""
    smallest = -1
    maxim = 0
    for word in test_range_dict:
        sorted_pos[word] = sorted(test_range_dict[word]['pos'])
        sorted_pointers[word] = 0
        if(smallest == -1 or sorted_pos[word][0] < smallest ):
            smallest = sorted_pos[word][0]
            smallest_word = word
        if(sorted_pos[word][0] > maxim ):
            maxim = sorted_pos[word][0]
            

        
    _range =  maxim - smallest
    
    if(sorted_pointers[smallest_word] == len(sorted_pos[smallest_word]) - 1):
        return _range
    
    sorted_pointers[smallest_word] = sorted_pointers[smallest_word] + 1
    smallest = sorted_pos[smallest_word][sorted_pointers[smallest_word]]
    
    while(True):
        for word in test_range_dict:
            ptr = sorted_pointers[word]
            if(sorted_pos[word][ptr] < smallest):
                smallest = sorted_pos[word][ptr]
                smallest_word = word
            if(sorted_pos[word][ptr] > maxim ):
                maxim = sorted_pos[word][ptr]
                
        if(sorted_pointers[smallest_word] >= len(sorted_pos[smallest_word]) - 1):
            if((maxim - smallest) <= _range):
                _range = maxim - smallest
            return _range
            
        if((maxim - smallest) <= _range):
            _range = maxim - smallest
        else:
            return _range
        sorted_pointers[smallest_word] = sorted_pointers[smallest_word] + 1
        smallest = sorted_pos[smallest_word][sorted_pointers[smallest_word]]
        
    return _range

In [37]:
# c = 2500

In [38]:
f_score = {}
for queryId in query_dict.keys():
    score = {}
    doc_terms = {}

    tfq = collections.Counter(query_dict[queryId])
    
    for term in query_dict[queryId]:
        if term in vocabmap.keys():
            term_id = vocabmap[term]
            data = texttojondecode(term_id)
            for doc in data['doc']:
                if doc not in score.keys():
                    score[doc] = 0
                    doc_terms[doc] = {}
                doc_terms[doc][term] = data['doc'][doc]
                
                _tf = int(data['doc'][doc]['tf']) 
                _l = doc_length_dict[doc]
                _term_1 = (totaldoc + 0.05) / (data['doc_freq'] + 0.5)
                _term_2 = ((1 + k1) * _tf) / (_tf + k1 * ((1-b) + b * ((_l/avg_len))))
                _term_3 = tfq[term] * ( 1 + k2) / (tfq[term] + k2)
                _score = log(_term_1) * _term_2 * _term_3 
                score[doc] = score[doc] + _score
                
    for doc in doc_terms.keys():
        _rangeOfWindow = findrange(doc_terms[doc])
        _numOfContainTerms = len(doc_terms[doc])
        _l = doc_length_dict[doc]
        omega = _rangeOfWindow / _numOfContainTerms
        pi = log(0.3 + exp(-omega))
        score[doc] = score[doc] + pi

    f_score[queryId] = sorted(score.items(), key=lambda kv: kv[1])[::-1][0:1000]


In [39]:
# file = open("proximity_scores", "w")
# for query in f_score.keys():
#     r = 1
#     for document in f_score[query]:
#         if(document[1] != 0):
#             file.write(str(query) + " Q0 " + reversedocmap[document[0]] + " " + str(r) + " " + str(document[1]) + " Exp\n" )
#         else:
#             print(query)
#         r = r + 1

In [40]:
# file = open("proximity_scores", "w")
# for query in f_score.keys():
#     r = 1
#     for document in f_score[query]:
#         if(document[1] != 0):
#             file.write(str(query) + " Q0 " + reversedocmap[document[0]] + " " + str(r) + " " + str(document[1]) + " Exp\n" )
#         else:
#             print(query)
#         r = r + 1

In [41]:
# len(doc_length_dict)

In [42]:
# C = 1500
# for doc in doc_terms.keys():
#     _rangeOfWindow = findrange(doc_terms[doc])
#     _numOfContainTerms = len(doc_terms[doc])
# #     _l = doc_length_dict[doc]

#     omega = (C - _rangeOfWindow) * _numOfContainTerms / (_l + V) 
# #     print(_rangeOfWindow / , exp(-_rangeOfWindow))
# #     break
# #     if(len(doc_terms[doc].keys()) > 2):
# #         print(doc_terms[doc])
# #         print(findrange(doc_terms[doc]))